In [ ]:
import vk_api
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

# Ваш токен доступа
ACCESS_TOKEN = 'vk1.a.fIyBnGPlHuga41SYQlhoNWOvASxz2bJLJl_F'

# Создание экземпляра VK API
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

# Создание пустого графа
G = nx.Graph()

# ID стартового пользователя
starting_user_id = '5487958'  # Замените на свой ID 233516504

# Максимальная глубина обхода
max_depth = 2

# Словарь для отслеживания цветов для каждого друга
friend_colors = {starting_user_id: 0}  # Добавляем стартового пользователя

def recursive_friends_search(user_id, depth, owner_id=None):
    if depth <= 0:
        return

    try:
        # Получение списка друзей пользователя
        friends = vk.friends.get(user_id=user_id, fields='id', count=1000)['items']

        for friend in tqdm(friends, desc="Обработка друзей"):
            friend_id = friend['id']
            G.add_edge(user_id, friend_id)
            
            # Определение цвета друга в зависимости от владельца
            if friend_id not in friend_colors:
                friend_colors[friend_id] = len(friend_colors)
            
            # Рекурсивный обход друзей друга
            recursive_friends_search(friend_id, depth - 1, friend_id)
    except vk_api.exceptions.VkApiError as e:
        print("Ошибка:", e)

# Запуск рекурсивного обхода для стартового пользователя
recursive_friends_search(starting_user_id, max_depth)

# Выделение компонент связности (кластеризация)
components = list(nx.connected_components(G))

# Визуализация графа с позиционированием и цветами
plt.figure(figsize=(40, 20))
pos = nx.spring_layout(G, seed=10, k=0.09)
for i, component in enumerate(components):
    subgraph = G.subgraph(component)
    node_colors = [friend_colors[node] for node in subgraph.nodes()]
    nx.draw_networkx_nodes(subgraph, pos, node_color=node_colors, cmap=plt.get_cmap('tab20'), node_size=100)
    nx.draw_networkx_edges(subgraph, pos)
plt.title("Граф друзей и друзей друзей")
plt.show()

In [ ]:
# Создание подграфа для друзей первого уровня
first_level_friends = list(G.neighbors(starting_user_id))
subgraph = G.subgraph([starting_user_id] + first_level_friends)

# Визуализация подграфа для друзей первого уровня
plt.figure(figsize=(40, 20))
pos = nx.spring_layout(subgraph, seed=10, k=0.09)
nx.draw(subgraph, pos, node_color='b', node_size=200, with_labels=False)
plt.title("Подграф друзей первого уровня для определенного пользователя")
plt.show()

In [ ]:
# Вывод количества вершин в подграфе
num_nodes = subgraph.number_of_nodes()
print("Количество вершин в подграфе:", num_nodes)